[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/google-ai-edge/model-explorer/blob/main/example_colabs/custom_data_overlay_demo.ipynb)

# Install packages

In [ ]:
# Install tflite & model-explorer.
!pip install tflite
!pip install --no-deps ai-edge-model-explorer ai-edge-model-explorer-adapter

# Install kagglehub (will be used in the next step to download a model)
!pip install kagglehub --no-deps

# Download MobileNet v3 from Kaggle

In [ ]:
import kagglehub

# This demo uses MobileNet v3, but you can use other models as well
path = kagglehub.model_download("google/mobilenet-v3/tfLite/large-075-224-classification")
model_path = f"{path}/1.tflite"

# Run the model with test data

In [ ]:
import numpy as np
import tensorflow as tf

# Load the TFLite model and allocate tensors.
interpreter = tf.lite.Interpreter(model_path=model_path)
interpreter.allocate_tensors()

# Get input and output tensors.
input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()

# Generate random input data.
for input_detail in input_details:
  input_shape = input_detail['shape']
  input_data = np.array(np.random.random_sample(input_shape), dtype=input_detail['dtype'])
  interpreter.set_tensor(input_detail['index'], input_data)

# Run the model on random input data.
interpreter.invoke()

# Examine the output data (optional)
for output_detail in output_details:
  print(f"Output for {output_detail['name']}")
  output_data = interpreter.get_tensor(output_detail['index'])
  print(output_data)

# Prepare per-op benchmarking data for Model Explorer

In [ ]:
!mkdir -p /tmp/data
%cd /tmp/data

# In this example, we're using profiling data from Android's Benchmarking tools
# that has already been mapped (outside of this Colab) to the Model Explorer schema.

# You can overlay per-op data of your choice by following the instructions at
# https://github.com/google/model-explorer/wiki/2.-User-Guide#custom-node-data

!wget -nc https://storage.googleapis.com/tfweb/model-explorer-demo/mv3-cpu-op-profile.json
!wget -nc https://storage.googleapis.com/tfweb/model-explorer-demo/mv3-xnnpack-op-profile.json
CPU_PROFILING_JSON_PATH="/tmp/data/mv3-cpu-op-profile.json"
XNNPACK_PROFILING_JSON_PATH="/tmp/data/mv3-xnnpack-op-profile.json"

# Visualize the model with per op latency

In [ ]:
import model_explorer

config = model_explorer.config()
(config
 .add_model_from_path(model_path)
 .add_node_data_from_path(CPU_PROFILING_JSON_PATH)
 .add_node_data_from_path(XNNPACK_PROFILING_JSON_PATH))

model_explorer.visualize_from_config(config)